# Step 1: Set up the LLM app

We're using a [Q&A example from LangChain](https://python.langchain.com/docs/use_cases/question_answering/quickstart) where we'll be doing a Q&A over a [blog](https://lilianweng.github.io/posts/2023-06-23-agent/).

In this notebook we setup an LLM and a RAG system via langchain. This is the "app" that we will be evaluating in the next few notebooks.

NOTE: This notebook and the following notebooks use OpenAI models to run code. You can set your API key in the `.env` file located in the root directory of this workshop's folder. Alternatively, you can use a different LLM either via API or locally by changing the code defining the LLM. We estimate that running all the notebooks together will cost $25 with OpenAI's models.

In [4]:
%reload_ext autoreload
%autoreload 2

import dotenv
dotenv.load_dotenv("../.env", override=True)

True

In [5]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from operator import itemgetter

In [6]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

# You can replace this with an LLM of your choice
# Refer: https://python.langchain.com/docs/integrations/llms/
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_to_string_list(docs):
    return [doc.page_content for doc in docs]

def concat_string(str_list):
    return "\n\n".join(str_list)

rag_chain = (
    {"context_list": retriever | format_to_string_list, 
     "context": retriever | format_to_string_list | concat_string, 
     "question": RunnablePassthrough()}
    | RunnablePassthrough()
    | {"answer": prompt | llm | StrOutputParser(), 
       "context": itemgetter("context_list")}
)

In [7]:
rag_chain.invoke("What is Task Decomposition?")

{'answer': 'Task Decomposition is a technique used to break down complex tasks into smaller and simpler steps. This approach helps agents to plan and execute tasks more efficiently by transforming big tasks into manageable subtasks. Task decomposition can be achieved through prompting techniques, task-specific instructions, or human inputs.',
 'context': ['Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.',
  'Tree of Thoughts (Yao et 